In [7]:
import kagglehub
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import tensorflow as tf

In [ ]:

print("Malignant, Tumor:")
glioma_folder ="Training/glioma"
x=0

for img_name in os.listdir(glioma_folder):
    img_path = os.path.join(glioma_folder, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Malignant, Tumor: " + str(x+1))
    plt.show()
    x+=1
    if(x>=5):
        break

In [ ]:
print("No tumor:")
notumor_folder="Training/notumor"
x=0

for img_name in os.listdir(notumor_folder):
    img_path = os.path.join(notumor_folder, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("No Tumor: " + str(x+1))
    plt.show()
    x+=1
    if(x>=5):
        break

In [ ]:
print("Beningn:")
benign_folder="Training/pituitary"
x=0

for img_name in os.listdir(benign_folder):
    img_path = os.path.join(benign_folder, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
    plt.title("Benign tumor: " + str(x+1))
    plt.show()
    x+=1
    if(x>=5):
        break

In [ ]:
original_folders = {
    "Glioma": "Training/glioma",
    "No Tumor": "Training/notumor",
    "Pituitary": "Training/pituitary",
    "Meningioma": "Training/meningioma"
}


categories = {
    "Tumor": ["Glioma", "Pituitary", "Meningioma"],
    "No Tumor": ["No Tumor"],
    "Benign": ["Pituitary"],
    "Malignant": ["Glioma", "Meningioma"]
}


category_counts = {cat: 0 for cat in categories}

for category, included_folders in categories.items():
    for folder in included_folders:
        folder_path = original_folders[folder]
        category_counts[category] += len(os.listdir(folder_path))

print("Number of images in each category:")
for category, count in category_counts.items():
    print(f"{category}: {count}")


image_formats = Counter()
image_sizes = Counter()

for folder, path in original_folders.items():
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)

        img = cv2.imread(img_path)

        if img is None:
            continue

        image_formats[img_name.split('.')[-1].lower()] += 1
        image_sizes[img.shape[:2]] += 1

print("\nImage formats:")
for fmt, count in image_formats.items():
    print(f"{fmt}: {count}")

print("\nImage sizes:")
for size, count in image_sizes.items():
    print(f"{size}: {count} images")

print("\nPotential Class Imbalance:")
max_class = max(category_counts, key=category_counts.get)
min_class = min(category_counts, key=category_counts.get)
print(f"Most images: {max_class} ({category_counts[max_class]} images)")
print(f"Fewest images: {min_class} ({category_counts[min_class]} images)")
imbalance_ratio = category_counts[max_class] / category_counts[min_class]
print(f"Imbalance ratio (max/min): {imbalance_ratio:.2f}")


In [4]:
original_folders = {
    "Glioma": "Training/glioma",
    "No Tumor": "Training/notumor",
    "Pituitary": "Training/pituitary",
    "Meningioma": "Training/meningioma"
}

resize_dim = (128, 128)

resized_normalized_images = []

for folder, path in original_folders.items():
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img_resized = cv2.resize(img, resize_dim)
        img_normalized = img_resized / 255.0
        resized_normalized_images.append(img_normalized)

print(f"Processed {len(resized_normalized_images)} images (resized and normalized).")


Processed 5712 images (resized and normalized).


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Path to the Training folder
folder_path = 'Training'  # Assuming all images are in the 'Training' folder, with subfolders

# Create lists to store images and labels
images = []
labels = []

# Dictionary to map subfolder names to labels (Tumor or No Tumor)
folder_labels = {
    "glioma": 1,
    "pituitary": 1,
    "meningioma": 1, 
    "notumor": 0 
}


for subfolder_name in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder_name)

    if os.path.isdir(subfolder_path):
        label_value = folder_labels.get(subfolder_name.lower())
        
        for img_name in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, img_name)

            if img_name.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                img = cv2.imread(img_path)
                if img is not None:
                    img_resized = cv2.resize(img, (128, 128)) 
                    img_normalized = img_resized / 255.0 

                    images.append(img_normalized)
                    labels.append(label_value)

x_data = np.array(images)
y_data = np.array(labels)

print(f"Shape of x_data: {x_data.shape}")
print(f"Shape of y_data: {y_data.shape}")

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.summary()


model.fit(x_data, y_data, epochs=10, batch_size=32)


model.evaluate(x_test, y_test, verbose=2)


Shape of x_data: (5712, 128, 128, 3)
Shape of y_data: (5712,)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_27 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,329 (49.36 MB)

 Trainable params: 12,939,329 (49.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 69s 355ms/step - accuracy: 0.7918 - loss: 0.4213
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 59s 332ms/step - accuracy: 0.9585 - loss: 0.1185
Epoch 3/10
 77/179 ━━━━━━━━━━━━━━━━━━━━ 33s 333ms/step - accuracy: 0.9688 - loss: 0.0873